In [ ]:
!pip install pandas-gbq==0.21.0

In [ ]:
"""
ETL PROFISSIONAL: Vendas de E-commerce (Simulador Nível #9)
Autor: Eric Pimentel | Mentor: Odin, O Pai dos Dados
"""

import pandas as pd
import numpy as np
import pandas_gbq
from google.auth import default
from google.colab import auth
from google.cloud import bigquery
from googleapiclient.discovery import build
import gspread
import kagglehub
import os
import logging
import warnings
warnings.filterwarnings("ignore", message="Parsing dates in DD/MM/YYYY format when dayfirst=True was specified")

# Configurar logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# =================================================================
# 1. EXTRAÇÃO
# =================================================================
def extract_data():
    """Extrai dados do Kaggle."""
    try:
        logging.info("🔎 Iniciando extração...")
        path = kagglehub.dataset_download("adarsh0806/influencer-merchandise-sales")
        files = os.listdir(path)

        for file in files:
            if file.endswith(".csv"):
                df = pd.read_csv(
                    os.path.join(path, file),
                    delimiter=',',  # Correção: delimiter=','
                    parse_dates=['Order Date']
                )
                logging.info("✅ Dados extraídos com sucesso!")
                return df

        raise FileNotFoundError("Nenhum CSV encontrado.")

    except Exception as e:
        logging.error(f"🚨 Erro na extração: {e}")
        return None

# =================================================================
# 2. TRANSFORMAÇÃO
# =================================================================
def transform_data(df):
    """Transforma e traduz os dados para PT-BR."""
    try:
        logging.info("🌀 Iniciando transformação...")

        # Tradução das colunas
        df = df.rename(columns={
            'Order ID': 'ID_Pedido',
            'Order Date': 'Data_Pedido',
            'Product Category': 'Categoria_Produto',
            'Buyer Gender': 'Genero_Comprador',
            'Buyer Age': 'Idade_Comprador',
            'Order Location': 'Localizacao_Pedido',
            'International Shipping': 'Envio_Internacional',
            'Sales Price': 'Preco_Venda',
            'Shipping Charges': 'Taxas_Envio',
            'Sales per Unit': 'Vendas_por_Unidade',
            'Quantity': 'Quantidade',
            'Total Sales': 'Vendas_Totais',
            'Rating': 'Avaliacao',
            'Review': 'Resenha'
        })

        # Limpeza e engenharia de features
        # --- Correção: Converter a coluna de data para string ---
        df['Data_Pedido'] = df['Data_Pedido'].dt.strftime('%Y-%m-%d %H:%M:%S')
        # -------------------------------------------
        df = df.drop_duplicates(subset=['ID_Pedido'])
        df['Margem_Lucro'] = df['Vendas_Totais'] - (df['Preco_Venda'] * df['Quantidade'] + df['Taxas_Envio'])
        df['Faixa_Etaria'] = pd.cut(df['Idade_Comprador'], bins=[0, 18, 25, 35, 100], labels=['<18', '18-25', '26-35', '35+'])

        logging.info("✅ Dados transformados!")
        return df

    except Exception as e:
        logging.error(f"🚨 Erro na transformação: {e}")
        return None

# =================================================================
# 3. CARREGAMENTO
# =================================================================
def load_data(df):
    """Carrega dados para Google Sheets e BigQuery."""
    try:
        logging.info("🚀 Iniciando carregamento...")
        auth.authenticate_user()

        credentials, _ = default()  # Correção: obter credenciais
        gc = gspread.authorize(credentials)  # Correção: autorizar o gspread

       # Google Sheets
        spreadsheet = gc.create('Vendas_Tratadas_Influenciador')
        spreadsheet.share(None, perm_type='anyone', role='writer')
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update([df.columns.tolist()] + df.values.tolist())
        logging.info("📊 Dados no Google Sheets: Prontos para o Looker!")

         # Configurar cliente do BigQuery
        client = bigquery.Client(project='projeto-vendas-influenciador')
        dataset_id = 'projeto_vendas'
        table_id = 'vendas_tratadas'

        # Criar ID completo da tabela (formato: projeto.dataset.tabela)
        full_table_id = f"{client.project}.{dataset_id}.{table_id}"

        # Carregar dados no BigQuery
        pandas_gbq.to_gbq(
            df,
            destination_table=full_table_id,
            project_id=client.project,
            if_exists='replace',  # Substitui a tabela se já existir
            progress_bar=False
        )
        logging.info("☁️ Dados no BigQuery: Prontos para SQL avançado!")

        return True

    except Exception as e:
        logging.error(f"🚨 Erro no carregamento: {e}")
        return False

# =================================================================
# 4. EXECUÇÃO PRINCIPAL
# =================================================================
if __name__ == "__main__":
    # Passo 1: Extrair
    dados_brutos = extract_data()

    if dados_brutos is not None:
        # Passo 2: Transformar
        dados_tratados = transform_data(dados_brutos)

        if dados_tratados is not None:
            # Passo 3: Carregar
            sucesso = load_data(dados_tratados)

            if sucesso:
                logging.info("🎉 Missão cumprida! Dashboard pronto no Looker!")
            else:
                logging.warning("⚠️ Concluído com erros parciais. Verifique os logs.")